In [19]:
using Random, Gadfly, Optim, Distributions, Distances, LinearAlgebra, ForwardDiff, Cairo

include("../iGMRF.jl");
include("../dataGen.jl");
include("../approxFramework.jl");
include("optim.jl");

In [20]:
# Seed
Random.seed!(400)
# Dimensions de la grille
m₁ = 3;
m₂ = 3;
# Nb total de cellules
m = m₁ * m₂;
# Matrice de structure
κᵤ = 100.0;
F = iGMRF(m₁, m₂, κᵤ);
# Grille cible
grid_target = generateTargetGrid(F);
grid_target[:, :, 1] = grid_target[:, :, 1] .+ 10;
# Génération de données
nobs = 100;
Y = generateData(grid_target, nobs);

In [21]:
res = KLOptim(F, Y)

 * Status: success

 * Candidate solution
    Final objective value:     -0.000000e+00

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    90
    f(x) calls:    157


## Analyse de l'approximation

In [22]:
η̂ = Optim.minimizer(res)[1:m+1];
L̂ = buildLowerTriangular(Optim.minimizer(res)[m+2:end], m);
Σ̂ = L̂*L̂'

d̂ = MvNormal(η̂, Σ̂);

In [30]:
η̂

10-element Vector{Float64}:
  5.925536731405483
 15.039497844919476
 10.003219857411954
  9.988501837312691
 10.011849314822804
  9.994964047076985
  9.989323934716568
  9.991383080168218
  9.981799145342451
  9.984572290323786

In [23]:
α = Optim.minimizer(initializeValues(θ -> densityTarget(θ, F=F, Y=Y), [1, fill(0.0, m)...]));
Σ₀ = round.(inv(computeFisherInformation(θ -> densityTarget(θ, F=F, Y=Y), α)), digits=5);
L₀ = cholesky(Σ₀).L;
L₀*L₀'

10×10 Matrix{Float64}:
  0.28061  -0.00329  -0.00201  0.00149  …  0.00106  0.0008   0.00301  0.00204
 -0.00329   0.00238   0.00133  0.00091     0.00079  0.00092  0.00076  0.00068
 -0.00201   0.00133   0.00198  0.00128     0.00097  0.00079  0.0008   0.00078
  0.00149   0.00091   0.00128  0.0023      0.00128  0.0007   0.0008   0.00092
 -0.00424   0.00137   0.00103  0.00078     0.00082  0.00129  0.00093  0.00077
 -9.0e-5    0.00099   0.00112  0.00097  …  0.0011   0.00097  0.00109  0.00097
  0.00106   0.00079   0.00097  0.00128     0.00194  0.00079  0.00097  0.00129
  0.0008    0.00092   0.00079  0.0007      0.00079  0.0023   0.00128  0.00091
  0.00301   0.00076   0.0008   0.0008      0.00097  0.00128  0.00195  0.00129
  0.00204   0.00068   0.00078  0.00092     0.00129  0.00091  0.00129  0.00231

In [25]:
Σ̂

10×10 Matrix{Float64}:
  0.280608  -0.003288  -0.002007  0.001495  …  0.000802  0.003009  0.002037
 -0.003288   0.002381   0.001333  0.000907     0.00092   0.000761  0.00068
 -0.002007   0.001333   0.001978  0.001279     0.000794  0.000801  0.000779
  0.001495   0.000907   0.001279  0.002302     0.0007    0.000798  0.000919
 -0.004237   0.00137    0.001029  0.000781     0.001291  0.000934  0.000769
 -9.3e-5     0.00099    0.001116  0.000971  …  0.000974  0.001094  0.000968
  0.001058   0.000789   0.000971  0.001283     0.000792  0.000974  0.001286
  0.000802   0.00092    0.000794  0.0007       0.002304  0.001279  0.000914
  0.003009   0.000761   0.000801  0.000798     0.001279  0.001947  0.00129
  0.002037   0.00068    0.000779  0.000919     0.000914  0.00129   0.002314

### Visuelle

Graphiques LOG

In [42]:
npoints = 10000; # Nombre de points sur lesquels évaluer la pdf
x = range(-10, 15, npoints);

# Mode de la posteriori
α = Optim.minimizer(initializeValues(θ -> densityTarget(θ, F=F, Y=Y), [1, fill(0.0, m)...]));

analysisSpace = zeros(m+1, npoints);
for j = 1:npoints
    analysisSpace[:, j] = [x[j], α[2:end]...];
end

simApprox = vec(mapslices(x -> logpdf(d̂, x), analysisSpace, dims=1));
simPosterior = vec(mapslices(x -> densityTarget(x, F=F, Y=Y), analysisSpace, dims=1));

p = plot(
    layer(x=x, y=simApprox, Geom.line, Theme(default_color="red")),
    layer(x=x, y=simPosterior, Geom.line),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Pour kappa_u")
)

draw(PNG("../plots/sqrt/log_kappa.png"), p)

In [43]:
npoints = 10000; # Nombre de points sur lesquels évaluer la pdf
x = range(-10, 18, npoints);

# Mode de la posteriori
α = Optim.minimizer(initializeValues(θ -> densityTarget(θ, F=F, Y=Y), [1, fill(0.0, m)...]));

analysisSpace = zeros(m+1, npoints);
for j = 1:npoints
    analysisSpace[:, j] = [α[1], x[j], α[3:end]...];
end

simApprox = vec(mapslices(x -> logpdf(d̂, x), analysisSpace, dims=1));
simPosterior = vec(mapslices(x -> densityTarget(x, F=F, Y=Y), analysisSpace, dims=1));

p = plot(
    layer(x=x, y=simApprox, Geom.line, Theme(default_color="red")),
    layer(x=x, y=simPosterior, Geom.line),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Pour mu")
)

draw(PNG("../plots/sqrt/log_mu.png"), p)

Approx vs MCMC

In [44]:
include("../mcmc.jl");

niter = 10000
δ² = 0.2
κᵤ₀ = 10
μ₀ = zeros(m)

θ = gibbs(niter, Y, m₁=m₁, m₂=m₂, δ²=δ², κᵤ₀=κᵤ₀, μ₀=μ₀);

warming_size = Int(0.2 * niter);
θsampling = θ[warming_size:end, :, :];

In [45]:
# Pour κᵤ

approxDensity(x::Real) = 1/x * pdf(Normal(η̂[1], Σ̂[1, 1]), log(x))
x = 0:1:1000

p = plot(
    layer(x=θsampling.value[:, 1, 1], Geom.histogram(density=true)),
    layer(x=x, y=approxDensity.(x), Geom.line, Theme(default_color="red")),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Approx vs MCMC pour kappa_u")
)

draw(PNG("../plots/sqrt/approxvsmcmc_kappa.png"), p)

In [46]:
# Pour μ

approxDensity(x::Real) = pdf(Normal(η̂[3], Σ̂[3, 3]), x)
x = 7:.1:13

p = plot(
    layer(x=θsampling.value[:, 2, 1], Geom.histogram(density=true)),
    layer(x=x, y=approxDensity.(x), Geom.line, Theme(default_color="red")),
    Guide.manual_color_key("Legend", ["Approximation", "Posteriori"], ["red", "deepskyblue"]),
    Theme(background_color="white"),
    Guide.title("Approx vs MCMC pour mu")
)

draw(PNG("../plots/sqrt/approxvsmcmc_mu.png"), p)